计算图
* 将代码分解成操作子
* 将计算表示成一个无环图
* 显示构造
  * Tensorflow/Theano/MXNet
* 隐式构造
  * PyTorch/MXNet

In [ ]:
# 显示构造
from mxnet import sym

a = sym.var()
b = sym.var()
c = 2 * a + b

In [ ]:
# 隐式构造
from mxnet import autograd, nd

with autograd.record():
    a = nd.ones((2, 1))
    b = nd.ones((2, 1))
    c = 2 * a + b

### 自动求导
假设我们想对函数$y=2\mathbf{x}^\top\mathbf{x}$关于列向量x求导

In [6]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

在我们计算y关于x的梯度之前，我们需要一个地穷来存储梯度。

In [7]:
x.requires_grad_(True) # 等价于 x.torch.array(4.0, requires_grads=True)
x.grad  # 默认值是None

现在让我们计算y
1. **点积计算**:
   - `torch.dot(x, x)` 计算向量 `x` 与自身的点积。对于一个向量 $ x = [x_1, x_2, \ldots, x_n] $，点积的计算公式为：
     $
     \text{dot}(x, x) = x_1^2 + x_2^2 + \ldots + x_n^2
     $
   - 这实际上是计算向量的平方和。

2. **乘以 2**:
   - 结果再乘以 2，即 $ y = 2 \times (x_1^2 + x_2^2 + \ldots + x_n^2) $。

In [8]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

通过调用反向传播函数来自动计算y关于×每个分量的梯度

- 在这个例子中，函数 $ y = x_1^2 + x_2^2 $ 的导数分别为 $ \frac{\partial y}{\partial x_1} = 2x_1 $ 和 $ \frac{\partial y}{\partial x_2} = 2x_2 $。
- 因此，当我们计算 $ y $ 对 $ x $ 的梯度时，得到的结果是：
  - 对于 $ x_1 = 2.0 $，梯度为 $ 4.0 $。
  - 对于 $ x_2 = 3.0 $，梯度为 $ 6.0 $。

In [9]:
y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

- `x.grad == 4 * x` 是一种验证方式，用于检查张量 `x` 的梯度是否与理论上预期的值相符。
- 在自动微分中，理解如何计算和验证这些梯度是非常重要的，它帮助您确保模型训练过程中的正确性。

In [ ]:
x.grad == 4 * x

现在让我们计算x的另一个函数。

In [10]:
# 在默认情况下，PyTorch会积累梯度，我们需要清除之前的值
x.grad.zero_()
y = x.sum()
y.backward()
x.grad

tensor([1., 1., 1., 1.])

深度学习中，我们的目的不是计算微分矩阵，而是批量中每个样本单独计算的偏导数之和

1. **反向传播**:
   - 由于 `y` 是一个非标量（例如，如果 `x` 是一个向量），直接调用 `y.backward()` 会导致错误。为了避免这个问题，我们通常对 `y` 的所有元素求和，然后调用 `.backward()`。
   - `y.sum()` 返回一个标量，这样就可以安全地调用 `.backward()`。在这种情况下，PyTorch 将计算 `y` 对 `x` 的梯度。

2. **查看梯度**:
   - 在调用 `.backward()` 后，`x.grad` 将包含 `y` 对 `x` 的梯度。在这个例子中，由于 $ y = x^2 $，根据链式法则：
     $
     \frac{dy}{dx} = 2x
     $
   - 因此，`x.grad` 将包含每个元素的值为 $ 2x $。
- 在 PyTorch 中，当对非标量调用 `.backward()` 时，需要传入一个 `gradient` 参数以指定微分函数。
- 使用 `.sum()` 方法将张量转换为标量是常见做法，这样可以安全地调用 `.backward()`。
- 确保在每次反向传播之前清零梯度，以避免累积旧的梯度值。

In [11]:
# d对非标量调用backward需要传入一个gradient参数，该参数指定微分函数
x.grad.zero_()
y = x * x
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward()
x.grad

tensor([0., 2., 4., 6.])

将某些计算移动到记录的计算图之外
* 在 PyTorch 中，您提供的代码段涉及到张量的梯度计算、张量的分离以及如何使用 `.backward()` 方法来计算梯度。以下是对每一行代码的详细解释，以及对 `x.grad == u` 的理解。
  - 在调用 `z.sum().backward()` 后，PyTorch 将根据链式法则计算出每个元素对 `z` 的贡献。
  - 因为 `z = u * x`，而且我们知道：
      $
      \frac{\partial z}{\partial x} = u
      $
  - 所以，最终得到的结果是：
      $
      x.grad = u
      $  
  - 因此，表达式 `x.grad == u` 将返回一个布尔值张量，表示每个元素是否相等。


- 在 PyTorch 中，通过 `.detach()` 创建的不需要梯度的张量可以用于避免影响计算图。
- 使用 `.backward()` 方法后，可以通过检查 `x.grad == u` 来验证计算出的梯度是否与预期相符。
- 此外，确保在每次反向传播之前清零旧的梯度，以避免累积错误。

In [12]:
x.grad.zero_()
y = x * x
u = y.detach()  # 把u当做常数
z = u * x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

In [13]:
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

即使构建函数的计算图需要通过Python控制流（例如，条件w"循环或任意函数调用），我们仍然可以计算得到的变量的梯度


- 在调用 `d.backward()` 后，PyTorch 会计算与输出张量 `d` 相对应的输入张量 `a` 的梯度，并将结果存储在 `a.grad` 中。
- 根据链式法则，假设我们有：
  - $ d = f(a) $

1. **如果最终选择了 `c = b`**:
   - 则 $ \frac{d}{da} = \frac{db}{da} $
   - 在这个情况下，由于 $ b = 2a $，所以 $ \frac{db}{da} = 2 $。
   - 因此，$ a.grad = d / a $ 是成立的。
2. **如果选择了 `c = 100 * b`**:
   - 则 $ \frac{d}{da} = 100 * \frac{db}{da} $
   - 在这种情况下，$ \frac{db}{da} = 2 $，所以 $ a.grad = 100 * 2 $。
   - 同样地，如果您检查 $ a.grad == d / a $，结果可能会根据具体情况而有所不同。

In [34]:
def f(a):
    b = a * 2
    while b.norm() < 1000: 
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

a.grad == d / a

tensor(True)